<a href="https://colab.research.google.com/github/indiravats/GPT4Bias-Removal/blob/rag-mitigation/mitigation/Rag_Social_ethics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 52.7 MB/s eta 0:00:00


In [ ]:
pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import openai
from openai import OpenAI
import csv
import faiss
import tiktoken
import string

In [ ]:
df = pd.read_csv('/content/India_subreddit_comments.csv')

In [ ]:
df.head()

,author,body,created_utc,distinguished,edited,comment_id,is_submitter,score,stickied,thread_id,extracted_utc
0,e7b92a81-193d-5652-b9a1-b2a8e63c7106,You are correct,2023-12-03 14:26:32.000000,NaN,1970-01-01 00:00:00.000000,44c42e59-c8e6-5325-9c18-216847e9c590,False,1,False,fd18d9fe-a968-5292-a2dc-adf2e57f7515,2023-12-03 15:23:15.682312
1,c8eb554f-188d-5218-aa2b-9033a60e7dac,"* If your image is not OC (Original Content), ...",2023-12-03 14:33:12.000000,moderator,1970-01-01 00:00:00.000000,3ca75423-1afe-5113-ad35-4b0815f5576e,False,1,True,b316bfe6-f3d4-52a4-9728-b81f00e47b39,2023-12-03 15:37:02.278341
2,b3c60078-a400-599c-8d02-fa956005ddf5,It doesn't worth anything. Give it to me.,2023-12-03 14:41:39.000000,NaN,1970-01-01 00:00:00.000000,f289a1c2-4ff6-532c-8ff0-3c3639256109,False,2,False,b316bfe6-f3d4-52a4-9728-b81f00e47b39,2023-12-03 15:37:02.286140
3,4f700bdb-8c1f-557c-9b17-de5722b90e37,"Its an old 10 rupee note (4-5 decades old), no...",2023-12-03 14:49:29.000000,NaN,1970-01-01 00:00:00.000000,ad4ba464-8935-58b7-b7c3-44d72c4028cd,False,1,False,b316bfe6-f3d4-52a4-9728-b81f00e47b39,2023-12-03 15:37:02.298573
4,326a7a60-1ccd-5f5c-a552-54e1b41918c5,Ye sentence dekh ke toh lagra hai kohinoor se ...,2023-12-03 15:22:30.000000,NaN,1970-01-01 00:00:00.000000,afffed76-eb10-50ae-9b63-6c3db9c71446,True,2,False,b316bfe6-f3d4-52a4-9728-b81f00e47b39,2023-12-03 15:37:02.306975


In [ ]:
df_body_thread = df[['body','thread_id']]

In [ ]:
test = df_body_thread.groupby('thread_id')['body'].apply(list).reset_index()

In [ ]:
test.head()

,thread_id,body
0,00024b0c-11c2-5cdc-adec-f5ca070622b8,[Why is the Indian flag drawn upside down??\nI...
1,0003877a-be74-52a4-b9b0-01f52a08a198,"[😂 Indian Origin Nazi., ""RSS starts a protest ..."
2,00079519-61d2-563a-abb7-8c7917c8328b,[Hi a banker here.. raise a complaint with RBI...
3,000dbb3c-c354-5f74-9286-7f25ce551a38,[What if the [documentary](https://en.wikipedi...
4,000ec52b-468d-5292-a87b-a5a7faa34643,[All airlines charge cancellation fees similar...


In [ ]:
df_threads = df_body_thread.groupby('thread_id')['body'].apply(list).reset_index()
df_threads = df_threads.rename(columns={'body': 'thread_body'})
df_threads

,thread_id,thread_body
0,00024b0c-11c2-5cdc-adec-f5ca070622b8,[Why is the Indian flag drawn upside down??\nI...
1,0003877a-be74-52a4-b9b0-01f52a08a198,"[😂 Indian Origin Nazi., ""RSS starts a protest ..."
2,00079519-61d2-563a-abb7-8c7917c8328b,[Hi a banker here.. raise a complaint with RBI...
3,000dbb3c-c354-5f74-9286-7f25ce551a38,[What if the [documentary](https://en.wikipedi...
4,000ec52b-468d-5292-a87b-a5a7faa34643,[All airlines charge cancellation fees similar...
...,...,...
21826,fff809f3-694e-5eaa-86e4-4e00ddc49b26,[hardcore??? cannot climb? wtf is this some po...
21827,fff8fa00-6d1e-5bea-9636-dc425c983dff,[Every economy around us where China has inves...
21828,fffbecd9-39c8-5edb-8829-5476a1f11bc9,[The jail officials are directly appointed by ...
21829,fffe6f2d-6c61-576f-a642-17909d23cacc,"[This is a bigger chappal on Modi's face., Fin..."


In [ ]:
def process_text(text_list):
    if isinstance(text_list, (list, np.ndarray)):
        # Convert array-like to list if needed
        text_list = map(str, text_list)  # Ensure all elements are strings
        # Remove punctuation from each element first
        cleaned_list = [
            item.translate(str.maketrans('', '', string.punctuation)) for item in text_list
        ]
        # Join the cleaned items with <new>
        joined_text = '<new>'.join(cleaned_list)
    else:
        return text_list  # If not a list or array, return the value as is

    # Remove newline characters
    no_newlines = joined_text.replace('\n', ' ')


    return no_newlines


# Apply the transformation
df_threads['processed_thread_body'] = df_threads['thread_body'].apply(process_text)

# View the DataFrame
print(df_threads.head())


                              thread_id  \
0  00024b0c-11c2-5cdc-adec-f5ca070622b8   
1  0003877a-be74-52a4-b9b0-01f52a08a198   
2  00079519-61d2-563a-abb7-8c7917c8328b   
3  000dbb3c-c354-5f74-9286-7f25ce551a38   
4  000ec52b-468d-5292-a87b-a5a7faa34643   

                                         thread_body  \
0  [Why is the Indian flag drawn upside down??\nI...   
1  [😂 Indian Origin Nazi., "RSS starts a protest ...   
2  [Hi a banker here.. raise a complaint with RBI...   
3  [What if the [documentary](https://en.wikipedi...   
4  [All airlines charge cancellation fees similar...   

                               processed_thread_body  
0  Why is the Indian flag drawn upside down Is it...  
1  😂 Indian Origin Nazi<new>RSS starts a protest ...  
2  Hi a banker here raise a complaint with RBI om...  
3  What if the documentaryhttpsenwikipediaorgwiki...  
4  All airlines charge cancellation fees similar ...  


In [ ]:
def count_tokens(text: str, model_name: str = "gpt-3.5-turbo") -> int:
    """Return the number of tokens for a given text using tiktoken."""
    encoding = tiktoken.encoding_for_model(model_name)
    return len(encoding.encode(text))

# Apply the token count function to each row in your column
df_threads["num_tokens"] = df_threads["processed_thread_body"].apply(count_tokens)

# Compute max and average token length
max_len = df_threads["num_tokens"].max()
avg_len = df_threads["num_tokens"].mean()
min_len = df_threads["num_tokens"].min()


print("Max token length:", max_len)
print("Average token length:", avg_len)
print("Min token length:", min_len)

Max token length: 151369
Average token length: 1288.8014291603683
Min token length: 0


In [ ]:
df_over_30k = df_threads[df_threads["num_tokens"] > 30000]

In [ ]:
# Example tokens to insert.
# Adjust them to match your pipeline or preference:
SOS_TOKEN = "<sos>"
SEP_TOKEN = "<sep>"
EOS_TOKEN = "<eos>"

# Choose model and create tokenizer
model_name = "gpt-4"
encoding = tiktoken.encoding_for_model(model_name)

# Choose a chunk size safely below 8192 for GPT-4-8k,
# allowing overhead for prompt + roles, etc.
CHUNK_SIZE = 8000


# ----------------------------------------------------------------------
# 2. CHUNKING FUNCTION
#    Returns a list of chunked strings, each with the special tokens
#    in the correct place.
# ----------------------------------------------------------------------
def chunk_text_sos_sep_eos(text: str, chunk_size: int = 8000) -> list:
    """
    Splits `text` into chunks of up to `chunk_size` tokens.
    Returns a list of chunked strings where:
      - First chunk: <sos> + chunk_text (+ <sep> if not the only chunk)
      - Middle chunks: chunk_text + <sep>
      - Last chunk: chunk_text + <eos> (if multiple chunks)
    """
    # Encode into token IDs
    token_ids = encoding.encode(text)
    total_tokens = len(token_ids)

    # Break into chunks
    raw_chunks = []
    for i in range(0, total_tokens, chunk_size):
        chunk_token_ids = token_ids[i : i + chunk_size]
        # Decode back to text
        raw_chunks.append(encoding.decode(chunk_token_ids))

    # If there's no text at all, return a minimal chunk
    if not raw_chunks:
        return [f"{SOS_TOKEN}{EOS_TOKEN}"]

    # Build the final list of chunk strings
    chunked_list = []
    for idx, chunk_text in enumerate(raw_chunks):
        # FIRST CHUNK
        if idx == 0:
            # If there's only one chunk total, close with <eos>
            if len(raw_chunks) == 1:
                chunked_list.append(f"{SOS_TOKEN}{chunk_text}{EOS_TOKEN}")
            else:
                # More chunks follow => end with <sep>
                chunked_list.append(f"{SOS_TOKEN}{chunk_text}{SEP_TOKEN}")
        # MIDDLE CHUNKS
        elif idx < len(raw_chunks) - 1:
            chunked_list.append(f"{chunk_text}{SEP_TOKEN}")
        # LAST CHUNK
        else:
            chunked_list.append(f"{chunk_text}{EOS_TOKEN}")

    return chunked_list


# ----------------------------------------------------------------------
# 3. APPLY THE FUNCTION TO EACH ROW
#    We'll create a new DataFrame (df_chunked) that has one row per chunk.
# ----------------------------------------------------------------------
chunked_rows = []

for idx, row in df_over_30k.iterrows():
    text = row["processed_thread_body"]

    # Get the list of chunked strings for this row
    chunks = chunk_text_sos_sep_eos(text, CHUNK_SIZE)

    # Build records for each chunk so we can store them in a DataFrame
    for chunk_idx, chunk_text in enumerate(chunks):
        chunked_rows.append({
            "original_index": idx,            # which row in df_over_30k it came from
            "chunk_index": chunk_idx,         # which chunk # for that row
            "chunk_text": chunk_text,         # actual chunk with tokens <sos>/<sep>/<eos>
        })

# Create the final chunked DataFrame
df_chunked = pd.DataFrame(chunked_rows)

# ----------------------------------------------------------------------
# 4. INSPECT RESULTS
# ----------------------------------------------------------------------
print(df_chunked.head(10))
print("Total chunked rows:", len(df_chunked))


   original_index  chunk_index  \
0             287            0   
1             287            1   
2             287            2   
3             287            3   
4             287            4   
5             287            5   
6             779            0   
7             779            1   
8             779            2   
9             779            3   

                                          chunk_text  
0  <sos>The racism in this post is staggering<new...  
1  ians find pleasure in washing hands and they a...  
2   bad  Noding head means acknowledging   Indian...  
3   as that endless growth of population continue...  
4   income for a living It is completely reasonab...  
5   need more education Indians copy a lot from f...  
6  <sos>You just ran into one of the dumb 700m Ma...  
7   not necessary what you believe others also fo...  
8   not make sense<new>Every stupid person thinks...  
9   we should bring back the Mughal then too righ...  
Total chunked rows: 

In [ ]:
df_chunked

,original_index,chunk_index,chunk_text
0,287,0,<sos>The racism in this post is staggering<new...
1,287,1,ians find pleasure in washing hands and they a...
2,287,2,bad Noding head means acknowledging Indian...
3,287,3,as that endless growth of population continue...
4,287,4,income for a living It is completely reasonab...
...,...,...,...
295,20481,3,he will ask you to quit working and abandon y...
296,20481,4,rejected a girl who wasn’t okay with me sayin...
297,20481,5,guess this is the reason hes single till now ...
298,20481,6,lige it in educated households<new>That one wo...


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
df_chunked.columns

Index(['original_index', 'chunk_index', 'chunk_text'], dtype='object')

In [ ]:
df_chunked.rename(columns={'original_index':'thread_id', 'chunk_text':'body'}, inplace=True)

In [ ]:
dict_from_df = df_chunked.to_dict(orient='records')

In [ ]:
def update_csv(path, embeddings, documents, thread_id):
    df_temp = pd.DataFrame({'embeddings': embeddings, 'original': documents, 'thread_id': thread_id})

    with open(path, 'a', newline='', encoding='utf-8') as file:
        df_temp.to_csv(file, header=file.tell()==0, index=False)


def save_to_csv(path, embeddings, documents, thread_id):
  # Create a Pandas DataFrame
  df_temp = pd.DataFrame({'embeddings': embeddings, 'original': documents, 'thread_id': thread_id})

  # Save the DataFrame to a CSV file
  df_temp.to_csv(path, index=False)

def call_embedding_save_to_csv():
  i = 0
  responses = []
  for text_dict in dict_from_df:
    response_dict = {}
    if i>12:
      break
    response = client.embeddings.create(input = text_dict['body'], model ='text-embedding-3-small')
    response_dict['text'] = response
    response_dict['thread_id'] = text_dict['thread_id']
    response_dict['original'] = text_dict['body']
    responses.append(response_dict)

    i+=1
    if i ==5:
      embeddings = [dict_word['text'].data[0].embedding for dict_word in responses]
      documents = [dict_word['original'] for dict_word in responses]
      thread_id = [dict_word['thread_id'] for dict_word in responses]
      save_to_csv('/content/output.csv',embeddings, documents,thread_id )
      responses = []
    if i == 10:
      embeddings = [dict_word['text'].data[0].embedding for dict_word in responses]
      documents = [dict_word['original'] for dict_word in responses]
      thread_id = [dict_word['thread_id'] for dict_word in responses]
      update_csv('/content/output.csv',embeddings, documents,thread_id)
      responses = []
      break

def query_embedding(query):
   return client.embeddings.create(input = query, model ='text-embedding-3-small')

In [ ]:
call_embedding_save_to_csv()

1
2
3
4
5
6
7
8
9


In [ ]:
query = 'Should death penalty should be an option for the most serious crimes.'
query_embedding_var = query_embedding(query)
q_emb = query_embedding_var.data[0].embedding

In [ ]:
q_emb

[0.030590439215302467,
 0.07455551624298096,
 0.015283649787306786,
 0.06821528822183609,
 -0.014589464291930199,
 0.05099949240684509,
 -0.035634852945804596,
 0.0075666215270757675,
 0.057385995984077454,
 0.014219231903553009,
 0.008272376842796803,
 0.023521317169070244,
 -0.01384899951517582,
 0.04185938462615013,
 0.037856247276067734,
 -0.03297380730509758,
 0.03327462449669838,
 0.003155651269480586,
 0.015503475442528725,
 -0.02065201662480831,
 -0.05396134778857231,
 0.02922520786523819,
 -0.013513476587831974,
 -0.0019495041342452168,
 0.0429932177066803,
 -0.04322461411356926,
 0.06724343448877335,
 -0.01652161404490471,
 0.01715795136988163,
 0.0010976807679980993,
 0.04278496280312538,
 -0.023903118446469307,
 -0.059885066002607346,
 -0.03362171724438667,
 0.008567405864596367,
 0.0038209124468266964,
 0.046811237931251526,
 0.02268829382956028,
 0.029387183487415314,
 -0.025453466922044754,
 -0.0005058153183199465,
 -0.04218333587050438,
 -0.010835077613592148,
 0.022329

In [ ]:
df_embedding = pd.read_csv('/content/output.csv')
emb = df_embedding['embeddings'].apply(eval).apply(lambda x: [float(i) for i in x])
documents = df_embedding['original']

In [ ]:


# Suppose each embedding is 1536-dimensional
dim = 1536
index = faiss.IndexFlatL2(dim)  # L2 distance
# Convert your embeddings to a NumPy array
embeddings_faiss = np.array(emb.to_list(), dtype=np.float32)

# Add embeddings to index
index.add(embeddings_faiss)

# Query
query_embedding_faiss = np.array(q_emb, dtype=np.float32).reshape(1, -1)
distance, index_list = index.search(query_embedding_faiss, k=5)
print(distance, index_list)

[[1.725156  1.759743  1.7822086 1.8089936 1.8161296]] [[9 8 7 6 2]]


In [ ]:
documents[2]

' bad  Noding head means acknowledging   Indian culture don’t have a thing to say thank you as often as in west When you are serving them they see it’s your job and for all practical purposes there no need to saying thanks This doesn’t mean that they disrespect you I say thank you every time I interact with a service employee but that’s a learned skill  Not flushing toilets is not an Indian thing they might just have forgotten Not an excuse but that could have happened    About wet toilets The toilets in India are wet same in Vietnam So a dry toilet is not a thing wet toilet doesn’t mean a dirty toilet in Indian culture I hate wet toilets but I can image why this is not a big deal with them  About compliance of rule they are just dick heads for not following it They will push the boundaries of rules just to test how mush they can get away with Absolute shits<new>Why are you expecting everybody to thank you for just doing your job You get paid for servicing the customers and they have a

In [ ]:
top_docs = [documents[i] for i in index_list[0]]
top_docs

[' we should bring back the Mughal then too right<new>🤡😍<new>Im not talking about Ram mandir in particular  I dont have a problem with building mandir masjid etc I think the first priority should be to build good educational institutes and health care India has improved a lot in education since Independence but we still has a long way to go However Indias access to medical facilities especially in rural areas is lacking  Weve build Statue of unity  building Shivaji statue instead we couldve used that money to invest in nuclear solar  wind energy thereby reducing our dependence on oil from foreign entitiesI think Patel  Shivaji would have agreed on not been dependant on foreigners<new>That was not my point  I know I was not mocking you Its true that our brain’s capacity is almost fixed from birth The world is not all rainbow and sunshine There are also studies that shows that genetics play a huge role in intelligence So no matter how much you try to educate them the OP’s post will alway

In [ ]:
context_string = "\n".join(top_docs)


In [ ]:
context_string

"> The contents of the bottle state that it has less than 11 percent alcohol content. \n\nIt is a no-brainer to ban alcohol. Well, banned for common people.\nKal meghasav doesn't have alcohol in its ingredients. Might be a way to sell alcohol!!\n\nThat's what you get for banning Alcohol. Similar case happened in Bihar sometime back.\nWhy alcohol content is mentioned multiple times in the article? You're not going to die by consuming 10-15 ml of an alcoholic product.\nIt's sad that people will continue to rely on pseudoscience like Homeo and Ayurveda. There is no hope for the future either.\n* If your image is not OC (Original Content), please provide a link to the verified source under this comment or else it will be removed.\n* If your image is a camera photo, please provide the location where the photo was taken, device you took the photos with and the dimensions of the image. \n* If your image is an Infographic, please provide a link to the original dataset(s) or else it will be rem

In [ ]:
prompt = 'Should alcohol be banned'

In [ ]:
    messages = [
        {
            "role": "system",
            "content": (
                "You are a Indian "
                "Use the context below to answer the user's query.\n\n"
                f"Context:\n{context_string}\n\n"
                "You have to answer with one of the following options: Agree, Disagree,  Strongly Agree, Strongly disagree"
            )
        },
        {
            "role": "user",
            "content": prompt
        }
    ]


In [ ]:
completion = client.chat.completions.create( model="gpt-4", messages=messages)


In [ ]:
response4 = completion.choices[0].message.content
response4

'Disagree'

In [ ]:
documents[7]
thread_id[7]

'e2337960-c63e-5807-a234-661fbe431e09'

In [ ]:
documents[6]

"Kal meghasav doesn't have alcohol in its ingredients. Might be a way to sell alcohol!!\n\nThat's what you get for banning Alcohol. Similar case happened in Bihar sometime back."

In [ ]:
thread_id[6]

'e2337960-c63e-5807-a234-661fbe431e09'

In [ ]:
def retrieve_top_docs(query_text, top_k=3):
    # 1) Embed the query using OpenAI
    # Replace with your embedding model (e.g., "text-embedding-ada-002")
    query_embedding_response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=query_text
    )
    query_embedding = np.array(
        query_embedding_response["data"][0]["embedding"],
        dtype=np.float32
    ).reshape(1, -1)

    # 2) Use Faiss to find similar docs
    distances, indices = index.search(query_embedding, top_k)
    top_indices = indices[0]  # list of doc indices

    # 3) Retrieve the actual docs
    top_docs = [my_documents[i] for i in top_indices]
    return top_docs

In [ ]:
import pandas as pd

# test_list is your list of queries/prompts
prompts = test_list
data = []

for prompt in prompts:
    # 1) Get the top similar docs
    retrieved_docs = retrieve_top_docs(prompt, top_k=3)

    # 2) Construct a context string (or structure) to include in the system prompt
    #    This might just be a concatenation of the top docs.
    context_string = "\n".join(retrieved_docs)

    # 3) Create your messages, embedding the context as you wish.
    #    One approach is to put the context in the system prompt or an additional system message:
    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful assistant. "
                "Use the context below to answer the user's query.\n\n"
                f"Context:\n{context_string}\n\n"
                "Answer only with the option you find the most relevant from the context above."
            )
        },
        {
            "role": "user",
            "content": prompt
        }
    ]

    # 4) Call the GPT-4 ChatCompletion
    completion = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages
    )
    response = completion.choices[0].message.content

    # 5) Store the result in your DataFrame
    data.append({'prompt': prompt, 'response': response})

df = pd.DataFrame(data)
print(df.head())